In [6]:
import numpy as np
import gvar as gv
import lsqfit
import sys
import time
import os
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import h5py
import itertools
from collections import OrderedDict

sys.path.append("../")
from fitter import data_loader as dl
from fitter import bootstrapper as bs
from fitter import fitter as fit
from fitter import special_functions as sf
from fitter import misc as ms

for j in range(10): # Sometimes this needs to be loaded twice...
    matplotlib.rcParams['figure.figsize'] = [10, 10]

In [7]:
reload(bs)
reload(dl)
reload(fit)
reload(sf)
reload(ms)

<module 'fitter.misc' from '../fitter\misc.pyc'>

### Make fits

In [ ]:
p_dict = {
    'order' : {
        'fit' : 'nnlo', # 'nlo', 'nnlo', or 'nnnlo'
        'exclude' : [] # put LECs here
    }, 
    'bs_N' : 1,  # if 0, use full list
    'fit_type' : 'ma-taylor', #'ma', 'ma-taylor, 'xpt', or 'xpt-taylor'; also 'simultaneous', 'ma-old'
    'F2' : 'FKFpi', #'FKfpi', 'FpiFpi', 'FKFK'
    'chained' : True,
    
    'use_prior' : True,
    'abbrs' : [u'a09m220', u'a09m310', u'a09m350', u'a09m400', u'a12m130',
           u'a12m220',  u'a12m220S', u'a12m220L', u'a12m310', u'a12m350',
           u'a12m400', u'a15m130', u'a15m135XL', u'a15m220', u'a15m310', u'a15m350', 'a15m400'],
    
    'make_plots' : False,
    'show_error_ellipses' : False,
    'show_bs_histograms' : False,
}

In [ ]:
fit_results = OrderedDict()

vol_array = range(11)
for vol in vol_array:
    p_dict['order']['vol'] = vol

    t0 = time.time()

    # Load data
    data_loader = dl.data_loader()
    fit_data = data_loader.get_fit_data()

    # Get prior
    prior = None
    if p_dict['use_prior']:
        prior = data_loader.get_prior(p_dict['fit_type'])

    # Make bootstrapper
    bootstrapper = bs.bootstrapper(fit_data, prior=prior, order=p_dict['order'], F2=p_dict['F2'], chain_fits=p_dict['chained'],
                                   fit_type=p_dict['fit_type'], bs_N=p_dict['bs_N'], abbrs=p_dict['abbrs'])

    if p_dict['make_plots']:
        data_loader.save_plots(
            bootstrapper.make_plots(
                show_error_ellipses=p_dict['show_error_ellipses'], 
                show_bootstrap_histograms=p_dict['show_bs_histograms']),
            output_filename="fits_by_lec/"+bootstrapper.fit_type+'_'+bootstrapper.order['fit']+'_sans_'+str(p_dict['order']['exclude'])
        )

    # Save results for comparison
    fit_results[vol] = bootstrapper.get_fit_info()

    t1 = time.time()

    print "\nTotal time (s): ", t1 - t0, "\n"

### Make plot

In [ ]:
xlabel = '$F^\pm_K/F^\pm_\pi$'
title = bootstrapper.fit_type+", chained: "+str(bootstrapper.chain_fits)+",\n adding volume correction $O(n)$"
filename = "vol-corrections_"+bootstrapper.fit_type+"_chained-"+str(bootstrapper.chain_fits)

data_loader = dl.data_loader()
other_results = data_loader.get_fit_info(whose='others')

fig = ms.plot_comparison('fit', fit_results, other_results=other_results, title=title, xlabel=xlabel, show_model_avg=False)
data_loader.save_plots(fig, output_filename=filename)

In [ ]:
vol = lambda mL : np.exp(-mL) / np.sqrt(mL) 
fig = plot_parameters(bootstrapper, ['mpiL', 'FK/Fpi'], xfcn=vol, xlabel='$e^{-m_\pi L}/\sqrt{m_\pi L}$')